# 1. Install and Import Dependencies

In [1]:
!pip install tensorflow==2.5.1
!pip install tensorflow-gpu==2.5.1
!pip install pixellib
!pip install opencv-python

  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.3.0
    Uninstalling typing-extensions-4.3.0:
      Successfully uninstalled typing-extensions-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
black 22.8.0 requires typing-extensions>=3.10.0.0; python_version < "3.10", but you have typing-extensions 3.7.4.3 which is incompatible.
  Using cached typing_extensions-4.3.0-py3-none-any.whl (25 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depend

In [2]:
import pixellib
from pixellib.instance import instance_segmentation
import cv2

2022-09-28 14:57:54.202913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rijul/miniconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-09-28 14:57:54.202950: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 2. Setup Model

In [3]:
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [6]:
segmentation_model = instance_segmentation()
segmentation_model.load_model('mask_rcnn_coco.h5')

2022-09-28 14:58:44.774857: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 14:58:46.181679: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2095929999 Hz


# 3. Get Inference

In [9]:
# Apply instance segmentation
def instSeg(ind):
    img_name = "raw_images/img ("+str(ind)+").jpg"
    frame = cv2.imread(img_name)
    res = segmentation_model.segmentFrame(frame, extract_segmented_objects = True)
    image = res[1]
    # print(res[0]['extracted_objects'])
    masks = res[0]['extracted_objects']
    areas = []
    # print(masks)
    for object_contours in masks:
        # print(object_contours.shape)
        positive_pixel_count = object_contours.astype(bool).sum() # assumes binary mask (True == 1)
        # h, w, l = object_contours.shape # assumes NHWC data format, adapt as needed
        # print(positive_pixel_count)
        h,w,l  = frame.shape
        area = positive_pixel_count / (w*h*l) * 100
        areas.append(area)
    # print(areas)
    areas_sum= round(sum(areas),2)
    print(areas_sum)
    img_out_name = "out/img ("+str(ind)+")_"+str(areas_sum)+".jpg"
    cv2.imwrite(img_out_name, image)



In [12]:
# for i in range(227,1,-6):
#     instSeg(i)

52.06
60.68
23.82
14.3
28.68
22.14
18.71


KeyboardInterrupt: 